In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [3]:
#df=df.drop(['AX', 'AY', 'AZ', 'BR', 'CB', 'CF', 'CW ', 'DN', 'EG', 'EL', 'EU', 'FC', 'FS', 'GB', 'GH', 'GI', 'Id'], axis =1 )

#val=val.drop(['AX', 'AY', 'AZ', 'BR', 'CB', 'CF', 'CW ', 'DN', 'EG', 'EL', 'EU', 'FC', 'FS', 'GB', 'GH', 'GI', 'Id'], axis =1 )


In [4]:
zero, one =np.bincount(df['Class'])

In [5]:
zero_df=df.loc[df['Class']==0]
one_df=df.loc[df['Class']==1]

In [6]:
zero_df=zero_df.dropna()
zero_df.shape

(446, 58)

In [7]:
one_df=one_df.fillna(df.median())
zero_df=zero_df.fillna(df.median())

/tmp/ipykernel_32/3900790800.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  one_df=one_df.fillna(df.median())
/tmp/ipykernel_32/3900790800.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  zero_df=zero_df.fillna(df.median())


In [8]:
zero_df=zero_df.drop(zero_df.sample(106).index)#

In [9]:
zero_df.shape

(340, 58)

In [10]:
data=pd.concat([one_df,zero_df])

In [11]:
data=data.sample(frac=1,random_state=42)
data = data.reset_index(drop=True)
#data=df
#data=data.fillna(data.median())

In [12]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [14]:
X=data.drop(["Class"], axis=1)
y=data["Class"]

In [15]:
'''def competition_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    #y_pred = y_pred.T[1]
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = (N_0 + N_1) / N_0
    w_1 = (N_0 + N_1) / N_1
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    sum_0 = -np.sum((1 - y_true) * np.log(p_0))
    sum_1 = -np.sum(y_true * np.log(p_1))
    to_output = ((w_0 / N_0) * sum_0 + (w_1 / N_1) * sum_1) / (w_0 + w_1)

    return to_output '''

'def competition_log_loss(y_true, y_pred):\n    # y_true: correct labels 0, 1\n    # y_pred: predicted probabilities of class=1\n    #y_pred = y_pred.T[1]\n    N_0 = np.sum(1 - y_true)\n    N_1 = np.sum(y_true)\n    w_0 = (N_0 + N_1) / N_0\n    w_1 = (N_0 + N_1) / N_1\n    # Calculate the predicted probabilities for each class\n    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)\n    p_0 = 1 - p_1\n    # Calculate the average log loss for each class\n    sum_0 = -np.sum((1 - y_true) * np.log(p_0))\n    sum_1 = -np.sum(y_true * np.log(p_1))\n    to_output = ((w_0 / N_0) * sum_0 + (w_1 / N_1) * sum_1) / (w_0 + w_1)\n\n    return to_output '

In [16]:
def competition_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    balanced_log_loss = (w_0 * log_loss_0 + w_1 * log_loss_1) / 2
    # return the average log loss
    return balanced_log_loss

In [17]:
from IPython.display import display
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold




def obj_xgb(trial, X,y):
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=1021)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.1, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.001, 0.2),
        'objective': 'binary:logistic', 
        'eval_metric': 'logloss', 
    }

    custom_scorer = make_scorer(competition_log_loss, greater_is_better=False)
    model = xgb.XGBClassifier(**params)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train,y_train)
        preds = model.predict_proba(X_test)[:,1]
        #scores = cross_val_score(model, x, y, cv=kf,scoring=custom_scorer)
        cv_scores[idx] = competition_log_loss(y_test, preds)
 
        avg=cv_scores.mean()

    
    #y_pred = model.predict_proba(x_valid)[:, 1]
    

    #roc_auc = roc_auc_score(y_valid, y_pred)
    
    return avg
    


study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: obj_xgb(trial, X, y), n_trials=100)  # Pass X and y to the obj_xgb function


display(study)

print('Best trial:', study.best_trial.params)
print('Best clogloss:', study.best_trial.value)

[I 2023-08-07 10:24:26,641] A new study created in memory with name: no-name-6c75075d-53bf-4ec0-8bd1-7dc900c1c619
/tmp/ipykernel_32/300164230.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.1, 1.0),
/tmp/ipykernel_32/300164230.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/tmp/ipykernel_32/300164230.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 

Best trial: {'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.14463868242527206, 'subsample': 0.9029714947424823, 'colsample_bytree': 0.5305435748830561, 'gamma': 0.015968888840893466}
Best clogloss: 0.28131809579227507


In [18]:
print('Best trial:', study.best_trial.params)
print('Best balanced_log_loss:', study.best_trial.value)

Best trial: {'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.14463868242527206, 'subsample': 0.9029714947424823, 'colsample_bytree': 0.5305435748830561, 'gamma': 0.015968888840893466}
Best balanced_log_loss: 0.28131809579227507


In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.15)

In [20]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

In [21]:
params ={'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.11083666084808587, 'subsample': 0.6646998689138404, 'colsample_bytree': 0.9993276251354577, 'gamma': 0.09161209891211826}

In [22]:
model = xgb.train(params, dtrain,300)

[10:29:24] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [23]:
y_pred = model.predict(dtest)

In [24]:
y_pred

array([ 4.01794970e-01, -3.18232663e-02,  6.11064136e-01,  8.89729321e-01,
        1.16453074e-01,  1.47869051e-01, -2.91372035e-02,  9.40503553e-02,
       -3.98443900e-02,  1.02560678e-02, -1.76302101e-02,  8.25492859e-01,
        8.52008443e-03,  2.51703203e-01,  4.40344632e-01,  3.81335855e-01,
        3.28621775e-01,  3.20440121e-02,  6.84981495e-02,  2.37309739e-01,
        7.33438432e-02,  1.22590043e-01, -1.39526092e-03,  1.11664444e-01,
       -2.99167540e-02,  7.38781273e-01,  5.47379971e-01,  2.95557693e-04,
        9.68140513e-02,  6.94513083e-01, -6.37892168e-03, -1.16434274e-02,
        2.99214602e-01,  1.97723329e-01,  5.61706431e-04,  2.97197439e-02,
       -2.67266389e-02,  4.30120751e-02,  1.03174222e+00, -3.35714892e-02,
        4.43718396e-02,  7.35686600e-01, -8.84786472e-02, -8.98632631e-02,
        3.61103922e-01,  1.45201668e-01,  4.89801317e-01,  1.62455328e-02,
        8.11784327e-01, -1.42795043e-02, -7.58324144e-03,  8.08672607e-02,
        1.04686081e-01,  

In [25]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions


y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.12%


In [26]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)

In [27]:
auc_score

0.9841269841269841

In [28]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred_binary)

# Extract true negatives (TN), false positives (FP), false negatives (FN), and true positives (TP) from the confusion matrix
TN, FP, FN, TP = confusion.ravel()

# Calculate the false positive rate (FPR) and false negative rate (FNR)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# Print the false positive rate and false negative rate
print("False Positive Rate (FPR):", FPR)
print("False Negative Rate (FNR):", FNR)

False Positive Rate (FPR): 0.018518518518518517
False Negative Rate (FNR): 0.21428571428571427


In [29]:
competition_log_loss(y_test,y_pred)

/tmp/ipykernel_32/4042287564.py:11: RuntimeWarning: divide by zero encountered in log
  log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))


0.2809749143480316

In [30]:
lb=np.zeros(val.shape[0])

In [31]:
dval = xgb.DMatrix(val, label=lb)


In [32]:
pred=model.predict(dval)

In [33]:
pred

array([0.4241249, 0.4241249, 0.4241249, 0.4241249, 0.4241249],
      dtype=float32)